# DReS-FL Experiment 

- K = 1
- T = 1
- input dimension = 1 -> scalar
- linear regression
- w = 1
- finite field size = 5
- inputs are selected from {0, 1}
- y = x + 1
- the gradient is supposed to be revealed at the end of each training round

Basically, our objective function is $l = (y - xw)^2$ and our gradient is $g = -2(xy - x^2w)$ 

In [ ]:
import numpy as np
from lcc.polynomials import LCCPoly

In [ ]:
prime = 5
data_range = 2
para_param = 1
priv_param = 1
w = 1

num_of_samples = 10
beta_arr = [0, 1]
alpha_arr = [2, 3, 4] # number of clients = 3 

In [ ]:
def create_encoded_dataset(encoded_data_pol, encoded_label_pol, alpha):
    encoded_dataset = np.empty((len(encoded_data_pol), 2)) 
    for idx, (data_pol, label_pol) in enumerate(zip(encoded_data_pol, encoded_label_pol)):
        encoded_dataset[idx][0] = data_pol(alpha)
        encoded_dataset[idx][1] = label_pol(alpha)
    return encoded_dataset

In [ ]:
secret_data = np.random.randint(low=0, high=2, size=(num_of_samples, 1))
secret_label = secret_data + 1
encoded_secret_data_pol = [LCCPoly(beta_arr, [x[0]], para_param, priv_param, prime) for x in secret_data]
encoded_secret_label_pol = [LCCPoly(beta_arr, [x[1]], para_param, priv_param, prime) for x in secret_label]